In [5]:
# Importació de llibreries per al processament de text,
# reducció de dimensionalitat, classificació i validació

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Càrrega del conjunt de dades pre-processat i reduït
# Es tracta d'un subconjunt del corpus Sentiment140 amb textos i etiquetes

df_reduït = pd.read_csv('twitter_reduït_Dataset.csv', encoding='latin-1')

# Separació de les variables predictives (textos) i de la variable objectiu (sentiment)
X = df_reduït['text'].values
y = df_reduït['target'].values

# Definició d’una llista de paraules buides personalitzades
# Aquestes paraules es filtraran durant la vectorització per no aportar informació rellevant

stopwords_personalitzades = [
    'as', 'an', 'the', 'in', 'on', 'at', 'to', 'of', 'and', 'or',
    'is', 'it', 'for', 'with', 'that', 'this', 'was', 'be',
    'are', 'were', 'been', 'from', 'by', 'about', 'into', 'out',
    'up', 'down', 'over', 'under', 'then', 'than', 'so', 'but', 'not'
]


In [7]:
# Paràmetres a explorar
valors_k = [10]
valors_C = [1]

# Inicialitzem els resultats
resultats = []

# Estratègia de validació creuada 
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Bucle sobre els diferents valors del paràmetre C
for C in valors_C:
    aucs = []

    for train_idx, test_idx in cv.split(X, y):
        texts_train = X[train_idx]
        texts_test = X[test_idx]
        y_train = y[train_idx]
        y_test = y[test_idx]

        # Vectorització TF-IDF ajustada només amb el conjunt d'entrenament
        vectoritzador = TfidfVectorizer(max_features=10000, stop_words=stopwords_personalitzades)
        X_train_vect = vectoritzador.fit_transform(texts_train)
        X_test_vect = vectoritzador.transform(texts_test)

        # Entrenament del model de regressió logística
        model = LogisticRegression(
            penalty='l2',
            C=C,
            solver='lbfgs',
            max_iter=100,
            random_state=42,
        )
        model.fit(X_train_vect, y_train)
        print("iteració")
        # Avaluació amb AUC-ROC
        probas = model.predict_proba(X_test_vect)[:, 1]
        auc = roc_auc_score(y_test, probas)
        aucs.append(auc)

    resultats.append({'C': C, 'auc_roc_mitjana': np.mean(aucs)})

# Resultats finals
df_resultats = pd.DataFrame(resultats)
print(df_resultats)


iteració
iteració
iteració
iteració
iteració
   C  auc_roc_mitjana
0  1         0.823919
